In [ ]:
# @title Download Checkpoint File and Install Python Packages
!pip install --no-cache-dir --upgrade \
  git+https://github.com/wkentaro/gdown.git \
  git+https://github.com/librosa/librosa.git \
  git+https://github.com/bmcfee/resampy.git \
  git+https://github.com/tastyForReal/piano_transcription_inference.git \
  git+https://github.com/yt-dlp/yt-dlp.git \
  git+https://github.com/spotDL/spotify-downloader.git \
  > '/dev/null' 2>&1

!gdown 1F3jUW2WQyIGbiPqUTcyE4zpd38v0sY7V

In [ ]:
# @title Create Form
import os
import shutil
import subprocess
import ipywidgets as widgets
from google.colab import files as colab_files
from datetime import datetime
from piano_transcription_inference import PianoTranscription, sample_rate, load_audio


def run_cmd(seq: list):
    str = subprocess.list2cmdline(seq)
    print(f'Running: "{str}"')
    subprocess.run(str)


def convert_to_wav():
    for filename in os.listdir('./downloaded'):
        input_path = os.path.join('./downloaded', filename)
        name, _ = os.path.splitext(filename)
        output_path = os.path.join('./downloaded', name + '.wav')
        run_cmd([
            'ffmpeg', '-i', input_path, '-ar', '16000', '-ac', '1', '-c:a',
            'pcm_s16le', output_path
        ])


download_method = widgets.ToggleButtons(
    options=['spotdl', 'yt-dlp'],
    description='Method:',
    disabled=False,
    button_style='',
)

input_text = widgets.Text(value='https://www.youtube.com/watch?v=VgX9bb425Cc',
                          description='Input:',
                          disabled=False)

convert_button = widgets.Button(
    description='Convert',
    disabled=False,
    button_style='',
)

display(download_method, input_text, convert_button)


def convert_music(btn):

    download_method_choice = download_method.value
    input_url = input_text.value

    convert_button.disabled = True
    print('Starting download process...')

    os.mkdir('downloaded')

    if download_method_choice == 'spotdl':
        run_cmd([
            'spotdl', input_url, '--output',
            './downloaded/{artists} - {title}/{artists} - {title}.{output-ext}'
        ])
    elif download_method_choice == 'yt-dlp':
        run_cmd(['yt-dlp', '-P', './downloaded'
                 '-f', 'bestaudio', input_url])
    else:
        print("Invalid download method selected.")

    midi_folder = datetime.utcnow().strftime('%Y-%m-%d_%H-%M-%S')
    os.mkdir(midi_folder)
    convert_to_wav()

    audio_files = [f for f in os.listdir('./downloaded') if f.endswith('.wav')]

    if not audio_files:
        print("No audio files found. Stopping script.")
        return

    print('Download complete. Starting conversion process...')

    for audio_file in audio_files:
        midi_file = datetime.utcnow().strftime('%Y-%m-%d_%H-%M-%S')
        midi_file_path = f'./{midi_folder}/{audio_file}_{midi_file}.mid'

        (audio, _) = load_audio(audio_file, sr=sample_rate, mono=True)

        transcriptor = PianoTranscription(
            device='cuda',
            checkpoint_path='./note_F1=0.9677_pedal_F1=0.9186.pth')
        _ = transcriptor.transcribe(audio, midi_file_path)

    if shutil.which('7z') is None:
        print("7z executable not found. Stopping script.")
        return
    archive_name = f"{midi_folder}.7z"
    run_cmd(["7z", "a", "-t7z", "-mx=9", archive_name, midi_folder])

    colab_files.download(archive_name)

    convert_button.disabled = False

    print(f"MIDI files created as {archive_name}")


convert_button.on_click(convert_music)